# Optimizing the number of staff in a chain of stores
[![staff_schedule.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/StaffChain/staff_schedule.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/mikhail/StaffChain/staff_schedule.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/StaffChain/staff_schedule.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/mikhail/StaffChain/staff_schedule.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/mikhail/StaffChain/staff_schedule.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/StaffChain/staff_schedule.ipynb)](https://colab.ampl.com)

## 1. Introduction

The presented optimization model aims to assign employees to pharmacies within a branch network, ensuring efficient staff allocation to meet customer demand while maximizing total profit. The model takes into account the following key elements:

* ***Pharmacies and Branches:*** The model organizes pharmacies within branches, considering their distinct working hours, distance between them, and daily sales and revenue data for different times of the day and year.

* ***Employee Assignment:*** Employees are assigned to pharmacies based on various constraints, including their base workplaces, maximum distance for reassignment, sales performance, and individual working hours or vacation schedules.

* ***Shift Types:*** Different shift types with specific attributes such as daily work hours, overtime allowances, and weekly or yearly working hour limits are defined. The model ensures that employees are not overworked and adhere to these limits.

* ***Staffing and Demand Management:*** The model incorporates the concept of peak hours and days, requiring additional staff to meet higher demand. A penalty is applied when the staffing level falls short of the expected sales demand.

* ***Revenue and Costs:*** The objective function maximizes the total profit by balancing pharmacy revenue, employee salary costs (including overtime), and penalties for unmet demand. The model dynamically adjusts for sales performance, ensuring that employees with higher performance are prioritized for assignment.

* ***Constraints:*** Multiple constraints are implemented to regulate the schedule, such as ensuring employees work only one continuous shift per cycle, do not exceed weekly or yearly overtime limits, and are assigned in accordance with their capacity to meet pharmacy sales demands.

The model optimizes employee allocation across a network of pharmacies, striving to meet customer demand while minimizing costs and ensuring compliance with labor regulations.

[*Partner with the AMPL team to transform complex problems into optimized solutions. AMPL consulting services combine deep technical knowledge with industry-leading insights, helping you unlock the full potential of optimization within your organization.*](https://ampl.com/services/)

Tags: mip, scheduling, data-driven model, amplpy, cbc, highs, gurobi

Notebook author: Mikhail Riabtsev <<mail@solverytic.com>>
***

## 2. Pharmacy Network Overview
There are historical data on sales of the pharmacy chain for the same period a year earlier. Available data include:

![alt text](https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/StaffChain/staff_schedule.jpg?raw=1)

### 1. Pharmacy Data

- **Branch Name**: The specific branch to which a pharmacy belongs, providing organizational context.
- **Pharmacy Address**: The physical location of each pharmacy, crucial for understanding regional performance and logistics.
- **Operating Days per Week**: The number of days each pharmacy is open, affecting sales potential and staffing needs.
- **Working Hours per Day**: The total hours each pharmacy is operational daily, influencing sales opportunities and shift planning.
- **Geotags**: Geographical coordinates of each pharmacy, useful for mapping and regional analysis.
Please review initial data [here](Pharmacies.csv).

### 2. Sales Data
- **Transaction Date and Time**: The exact timestamp of each sale, essential for analyzing peak sales periods and daily trends.
- **Pharmacy Location**: The specific pharmacy where the transaction took place, allowing for location-based sales analysis.
- **Salesperson**: The individual responsible for the sale, linking performance metrics to specific employees.
- **Items Sold**: The quantity of products sold during each transaction, critical for inventory management and sales tracking.
- **Profit Generated**: The profit margin on each sale, providing financial insights at both micro (per transaction) and macro (overall network) levels.
Please review initial data [here](Sales.csv).

### 3. Personnel Data
- **Hourly Wages**: The pay rate for employees, essential for calculating labor costs and analyzing payroll expenses.
- **Base Pharmacy**: The primary location where an employee is assigned, relevant for tracking employee movement and shift assignments.
- **Replacement Distance**: The maximum distance a worker can travel to temporarily work at a different pharmacy when needed.
- **SalesPerformance**: Employee sales performance for priority scheduling.
- **Vacation Preferences**: The preferred start dates for employee vacations, aiding in scheduling and ensuring adequate staffing during peak periods.
- **Vacation Duration**: Duration of vacation, days.
Please review initial data [here](Employees.csv).

## Work Methods

The network uses three distinct work methods, each designed to meet different operational needs and governed by specific rules:

### 1. Base Work Method
- **Daily Work Hours**: Employees work standard 8-hour shifts.
- **Weekly Work Hours**: Total work hours per week are capped at 40.
- **Overtime Rules**: Employees can work up to 2 and 8 additional hours per day and week respectively, with an annual limit of 96 hours. Overtime is compensated at double the regular wage.

### 2. Rotational Work Method
- **Shift Duration**: Shifts can be up to 12 hours long.
- **Work Schedule**: Employees work for 2 consecutive days, followed by 2 days off.

### 3. Additional Work Method
- **Work Hours**: Employees can work up to 4 hours per day, accommodating those seeking part-time roles.
- **Overtime Rules**: Employees can work up to 2 and 8 additional hours per day and week respectively, with an annual limit of 96 hours. Overtime is compensated at double the regular wage.

## General Rules
The pharmacy network adheres to several overarching rules designed to maintain a fair and efficient work environment:

- **Weekly Workdays**: Employees are limited to working a maximum of 6 days per week to prevent burnout.
- **Vacation Entitlement**: Employees are entitled to n days of leave per year.
- **Schedule Flexibility**: Work schedules can only be adjusted every 14 days to maintain stability in staffing and operations.
- **Peak Demand**: During peak hours (12PM - 20PM) and peak season (4-5 months of the year), staff must be assigned to meet demand equal to the available data +10%.

## Objective

Optimize the pharmacy chain's workforce to maximize profitability while adhering to regulatory and operational standards. Ensure each branch operates efficiently by balancing key factors such as employee well-being, customer demand, and logistical challenges. Develop a sustainable and flexible operating strategy that supports long-term success.
***

Let's start by downloading the necessary extensions and libraries

In [1]:
# Install dependencies
%pip install -q amplpy pandas
import pandas as pd
import numpy as np

In [2]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["cbc", "highs", "gurobi"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

## 3. AMPL formulation of the model

Use %%writefile to create a file with the model

In [3]:
%%writefile staff_schedule.mod
reset;

### SETS
  set BRANCH ;                                        # Set of all branches where pharmacies are located
  set PHARM;                                          # Set of all pharmacies
  set PHARMACIES{BRANCH} within PHARM;                # Set of pharmacies within each branch
  set EMPLOYEES ;                                     # Set of employees
  set EMPL_WORKPLACE{EMPLOYEES} within PHARM;         # Set of base pharmacies for each employee, indicating their usual workplace
  set SHIFT_TYPES ;                                   # Set of shift types available for assignment

  param nYearDayData = 31 ;                           # Number of days for which sales and staff data are available
  param nYearDay = nYearDayData ;                     # Number of days under consideration in the current optimization model
  param SchedCycle = nYearDayData ;                   # The number of days in a scheduling cycle.
                                                      # Frequency at which an employee can switch between different shift modes, pharnacies.
                                                      # Here we can set up different conditions for different [shifts, pharmacies]
  param nScheduleCycle = ceil(nYearDay/SchedCycle) ;  # Number of scheduling cycles in the model period
  param nWeekDay = 7 ;                                # Number of days in a week
  param nWeek = ceil(nYearDay/nWeekDay) ;             # Total number of weeks considered in the model
  param nHour = 23 ;                                  # Number of hours in a working day (0 to 23, assuming 24-hour coverage)

  set PHARM_HOURS_DATA = {PHARM, 1..nYearDayData, 0..nHour};  # Set of valid combinations of pharmacies, days, and hours from the dataset
  set SHIFT_HOURS = {PHARM, SHIFT_TYPES, 0..nHour, 0..nHour}; # Set of possible pharmacy, shift, and hour combinations
  set PEAK_HOURS = {12..20} ;                         # Hours of the day when demand peaks (e.g., lunch and evening)
  set PEAK_DAYS = {120..180} ;                        # Peak days with high demand, often due to seasonal trends


### PARAMETERS
 ## Pharmacy-specific parameters
  param ph_StartTime{PHARM} >= 0 ;                    # Start time of the workday at each pharmacy (in hours, 0..23)
  param ph_WorkDuration{PHARM} >= 0 ;                 # Duration of the working day for each pharmacy (in hours)
  param ph_Distance{p in PHARM, pp in PHARM: p != pp} >= 0 ;  # Distance between pharmacies
  param staffReserve >= 0 ;                           # Reserve staff requirement during peak times (extra staff needed during busy hours)

  param ph_ItemsSold{PHARM_HOURS_DATA} default 1 >= 0;# Number of items sold at each pharmacy at each hour on each day
  param ph_AdjustedItemsSold{(ph,d,h) in PHARM_HOURS_DATA} := # Adjusted number of items sold during peak times, with a staff reserve multiplier
   if (h in PEAK_HOURS or d in PEAK_DAYS)
   then ph_ItemsSold [ph,d,h] * staffReserve          # Multiply sales during peak hours/days by the reserve factor
   else ph_ItemsSold [ph,d,h] ;                       # Regular sales during non-peak times

  param ph_Revenue{PHARM_HOURS_DATA} default 0 >= 0 ; # Revenue generated from sales at each pharmacy for each hour on each day
  param penaltyCoeff >= 0 default 1;                  # Penalty coefficient for unmet demand

  param ph_DaySchedule {SHIFT_HOURS} >= 0 default 0 ; # Defines pharmacy work schedules per day for different shifts


  ## Shift-related parameters
  set SHIFT_PARAM = {                                 # Set of different parameters defining shift rules and limitations.
    'DailyWorkHours',                                 # Working hours per day by shift type
    'OvertimeHours',                                  # Overtime hours per day allowed for shift type.
    'maxWeeklyHours',                                 # Maximum working hours per week by shift type
    'maxWeeklyOvertimeHours',                         # Maximum overtime hours per week by shift type
    'maxYearOvertimeHours',                           # Maximum overtime hours per year by shift type
    'TotalHours'};                                    # Total hours per shift, including both regular and overtime hours

  param shiftAtr {SHIFT_TYPES, SHIFT_PARAM} >= 0 ;    # Defines parameters for each shift type (e.g., daily hours, overtime)
  param cycleSchedule{SHIFT_TYPES, 1..SchedCycle} >= 0 default 0; # Schedule defining how shifts are distributed across the days

 ## Employee-specific parameters
  set EMPL_PARAM = {                                  # Set of parameters related to employee attributes.
    'MaxItemsSoldPerHour',                            # Max number of items an employee can sell per hour (workload capacity)
    'MaxReplaceDistance',                             # Maximum distance between pharmacies for employee reassignment
    'SalaryPerHour',                                  # Hourly wage for each employee
    'VacationStartPeriod',                            # Start day for vacations for each employee
    'VacationDuration',                               # Duration of vacation for each employee
    'SalesPerformance'} ;                             # Sales performance metric for each employee (used for priority scheduling)

  param emplAtr{EMPLOYEES, EMPL_PARAM} >= 0 ;         # Table of employee-specific attributes and constraints.

  # Total salary cost for each employee per shift, including overtime.
  param empl_SalaryPerShift{e in EMPLOYEES, sh in SHIFT_TYPES} =
    emplAtr[e, 'SalaryPerHour'] * (shiftAtr[sh, 'DailyWorkHours'] + 2 * shiftAtr[sh, 'OvertimeHours']) ;

 ## Assignment Links and Shift Links Definitions
  # ASSIGNMENT_LINKS defines valid assignment scenarios of employees to branches, pharmacies, days, and hours, based on certain conditions.
  set ASSIGNMENT_LINKS within {br in BRANCH, PHARMACIES[br], EMPLOYEES, 1..nYearDay, 0..nHour} :=
    setof {br in BRANCH, ph in PHARMACIES[br], e in EMPLOYEES, d in 1..nYearDay, h in 0..nHour:
    (ph, d, h) in PHARM_HOURS_DATA and                # Only consider valid working hours
    d >= 1 and d <= nYearDay and                      # Only consider valid working days
    h >= ph_StartTime[ph] and                         # Ensure the hour is after or at the start of the working day
    h < ph_StartTime[ph] + ph_WorkDuration[ph]-1 and  # Ensure the hour is before the end of the working day
    sum {ew in EMPL_WORKPLACE[e]} (if (ph != ew) then ph_Distance[ew, ph] else 0) <= emplAtr[e, 'MaxReplaceDistance'] and # Ensure max distance constraint is respected
    ph in PHARMACIES[br] and                          # Pharmacy must be within the current branch
    exists {ew in EMPL_WORKPLACE[e]} (ew in PHARMACIES[br]) and     # Employee's base pharmacy must be in the same branch
    not (d >= emplAtr[e,'VacationStartPeriod'] and d < emplAtr[e,'VacationStartPeriod'] + emplAtr[e,'VacationDuration']) # Exclude vacation period
    } (br,ph,e,d,h);                                  # Set of valid assignments based on above conditions

  # Set of valid branch-pharmacy-employee-day-hour-shift type combinations
  set SHIFT_LINKS within {br in BRANCH, PHARMACIES[br], EMPLOYEES, 1..nYearDay, 0..nHour, SHIFT_TYPES} :=
    setof {(br,ph,e,d,h) in ASSIGNMENT_LINKS,  sh in SHIFT_TYPES: # Iterate over valid assignment links and shift types to determine valid shift start times
    h <= ph_StartTime[ph]                             # Ensure that the shift can start and finish within the pharmacy's working hours. The shift must end before the pharmacy closes
    + ph_WorkDuration[ph]
    - shiftAtr[sh,'TotalHours']}
    (br,ph,e,d,h,sh);                                 # Creates the set of valid shift links based on the conditions above.

  # Set for valid assignments considering branch, pharmacy, employee, day, and shift
  set SHIFT_LINKS_br_ph_e_d_sh  := setof{(br,ph,e,d,h,sh) in SHIFT_LINKS}   (br,ph,e,d,sh);
  set SHIFT_LINKS_br_ph_e_sh    := setof{(br,ph,e,d,h,sh) in SHIFT_LINKS}   (br,ph,e,sh);
  set SHIFT_LINKS_e_sh          := setof{(br,ph,e,d,h,sh) in SHIFT_LINKS}   (e,sh);
  set LINKS_br_ph_d_h           := setof{(br,ph,e,d,h) in ASSIGNMENT_LINKS} (br,ph,d,h) ;
  set LINKS_e_d                 := setof{(br,ph,e,d,h) in ASSIGNMENT_LINKS} (e,d) ;

### VARIABLES
  # Binary variable for the start of an assignment of employees to pharmacies and shift
  var StartPeriodSched{(br,ph,e,sh) in SHIFT_LINKS_br_ph_e_sh, 1..nScheduleCycle} binary ;

  # Binary variable indicating if an employee is assigned to work during a specific hour
  var StartDaySchedule{(br,ph,e,d,h,sh) in SHIFT_LINKS} binary;

  # Binary variable indicating the scope & conditions of assign shift by days
  var AssignDaySchedule{(br,ph,e,d,h) in ASSIGNMENT_LINKS} =
    sum{(br,ph,e,d,t,sh) in SHIFT_LINKS: t >= h - shiftAtr[sh,'TotalHours'] and t <= h}
    StartDaySchedule[br,ph,e,d,t,sh] * ph_DaySchedule[ph,sh,t,h] ;

  var StaffShortageSlack{LINKS_br_ph_d_h} >= 0 ;      # Variable representing a slack variable used when there's a staff shortage

### OBJECTIVE FUNCTION  # Objective function to maximize total profit by balancing pharmacy revenue, employee costs, and penalties for unmet demand
maximize TotalProfit:                                 # Objective function aiming to maximize total profit by balancing pharmacy revenue, employee costs, and penalties for unmet demand.
    sum{(br,ph,d,h) in LINKS_br_ph_d_h}
    (ph_Revenue[ph,d,h] * (1 - StaffShortageSlack[br,ph,d,h] / ph_ItemsSold[ph,d,h])  # Revenue from sales adjusted by the minimum demand.
    - penaltyCoeff * StaffShortageSlack[br,ph,d,h])               # Penalties for staff shortages.
    - sum{(br,ph,e,d,t,sh) in SHIFT_LINKS}
    StartDaySchedule[br,ph,e,d,t,sh] * empl_SalaryPerShift[e,sh] ;# Employee salary costs


### CONSTRAINTS
  # 1. Ensure that employees only start one shift per scheduling cycle (SchedCycle)
  s.t. PeriodCycle {e in EMPLOYEES, sch in 1..nScheduleCycle}:
    sum {(br,ph,e,sh) in SHIFT_LINKS_br_ph_e_sh}
    StartPeriodSched[br,ph,e,sh,sch] <= 1;

  # 2. Ensure valid shift assignments are respected
  s.t. ShiftAssignValidity {(br,ph,e,d,sh) in SHIFT_LINKS_br_ph_e_d_sh}:
    sum{(br,ph,e,d,h,sh) in SHIFT_LINKS} StartDaySchedule[br,ph,e,d,h,sh] <=
    sum{sch in 1..nScheduleCycle: sch = ceil(d/SchedCycle)}
      StartPeriodSched[br,ph,e,sh,sch] * cycleSchedule[sh,d] ;

  # 3. Ensure enough staff is available to handle the expected sales and Reserve of staff at each pharmacy
  s.t. StaffNumber{(br,ph,d,h) in LINKS_br_ph_d_h}:
    sum{(br,ph,e,d,h) in ASSIGNMENT_LINKS}
      AssignDaySchedule[br,ph,e,d,h] * emplAtr[e,'MaxItemsSoldPerHour'] #
      >= ph_AdjustedItemsSold[ph,d,h] - StaffShortageSlack[br,ph,d,h] ;

 ## Hours Constraints:
  # 4. Ensure employees do not exceed weekly working hours
  s.t. WeekHourLimit {(e,sh) in SHIFT_LINKS_e_sh, w in 1..nWeek}:
    sum{(br,ph,e,d,h,sh) in SHIFT_LINKS: d >= (w-1) * nWeekDay + 1 and d <= w * nWeekDay}
    StartDaySchedule[br,ph,e,d,h,sh] * shiftAtr[sh,'TotalHours']
    <= shiftAtr[sh,'maxWeeklyHours'] ;

  # 5. Ensure that weekly overtime hours do not exceed the maximum allowed
  s.t. WeekOvertimeLimit {e in EMPLOYEES, w in 1..nWeek, sh in SHIFT_TYPES}:
    sum{(br,ph,e,d,h,sh) in SHIFT_LINKS: d >= (w-1) * nWeekDay + 1 and d <= w * nWeekDay}
    StartDaySchedule[br,ph,e,d,h,sh] * shiftAtr[sh,'OvertimeHours']
    <= shiftAtr[sh,'maxWeeklyOvertimeHours'] ;

  # 6. Ensure that yearly overtime hours do not exceed the maximum allowed
  s.t. YearOvertimeLimit {e in EMPLOYEES, sh in SHIFT_TYPES}:
    sum{(br,ph,e,d,h,sh) in SHIFT_LINKS}
    StartDaySchedule[br,ph,e,d,h,sh] * shiftAtr[sh,'OvertimeHours']
    <= shiftAtr[sh,'maxYearOvertimeHours'] ;

Writing staff_schedule.mod


In [4]:
# Read the model
ampl.read('staff_schedule.mod')

## 4. Load data

### 4.1. Load data from Sales.csv

In [5]:
sales_file = 'https://raw.githubusercontent.com/ampl/colab.ampl.com/refs/heads/master/authors/mikhail/StaffChain/Sales.csv'
pharmacies_file = "https://raw.githubusercontent.com/ampl/colab.ampl.com/refs/heads/master/authors/mikhail/StaffChain/Pharmacies.csv"
employees_file = "https://raw.githubusercontent.com/ampl/colab.ampl.com/refs/heads/master/authors/mikhail/StaffChain/Employees.csv"

# The 'keep_default_na' parameter is set to False to avoid automatic replacement of missing values with NaN.
df = pd.read_csv(sales_file, delimiter = ";", keep_default_na = False)

# Define a set in AMPL named 'PHARM', containing unique shop names from the 'Shop' column in the DataFrame.
ampl.set['PHARM'] = set(df['Shop'])
print(set(df['Shop']))
# Define a set in AMPL named 'LINKS_' that contains tuples of ('Shop', 'DayOfYear', 'TimePeriod').
# Each row in the DataFrame is used to create these tuples.
#ampl.set['PHARM_HOURS_DATA'] = df.apply(lambda row: (row['Shop'], row['DayOfYear'], row['TimePeriod']), axis=1)

# Get a list of columns from the DataFrame, excluding 'ph_Revenue' and 'ph_ItemsSold'.
# This list will be used as the index when setting parameters in AMPL.
set_of_columns = df.loc[:, (df.columns != "pharm_Revenue") & (df.columns != "pharm_ItemsSold")].columns.tolist()

# Iterate over the list of numeric columns to replace commas with dots and convert the data type to float.
#numeric_columns = ['pharm_Revenue']  # List of columns to convert to numeric
#for column in numeric_columns:
#   df[column] = df[column].str.replace(',', '.').astype(float)

# If there are columns left to index (i.e., the DataFrame is not empty after excluding 'ph_Revenue' and 'ph_ItemsSold'),
# set the 'ph_Revenue' and 'ph_ItemsSold' parameters in AMPL, using the remaining columns as the index.
if set_of_columns:                                                              # If there are any remaining columns
    ampl.param["ph_Revenue"] = df.set_index(set_of_columns)["pharm_Revenue"]
    ampl.param["ph_ItemsSold"] = df.set_index(set_of_columns)["pharm_ItemsSold"]

{33, 34, 35, 36, 37, 38, 47, 48}


### 4.2.1. Load data from Pharmacies.csv

In [6]:
df = pd.read_csv(pharmacies_file, delimiter = ";", keep_default_na = False)
# Define the sets
# # Define a set in AMPL named 'BRANCH', containing unique branch names from the 'BranchName' column in the DataFrame.
ampl.set['BRANCH'] = set(df['BranchName'])

# Group the DataFrame by the 'BranchName' column and for each branch, collect the corresponding 'Pharmacy_id' values into a set.
# The 'apply(set)' function converts values into sets for each branch.
# The 'to_dict()' method then converts this Series of sets into a dictionary.
# In this dictionary, the keys are branch names and the values are sets of pharmacy IDs.
ampl.set['PHARMACIES'] = df.groupby('BranchName')['Pharmacy_id'].apply(set).to_dict()

# Define parameters in AMPL related to pharmacy working hours.
# 'ph_StartTime' is set with 'Pharmacy_id' as the index and 'OpenTime' as the value.
ampl.param["ph_StartTime"] = df.set_index('Pharmacy_id')["OpenTime"]
# 'ph_WorkDuration' is set with 'Pharmacy_id' as the index and 'Work period, hours' as the value.
ampl.param["ph_WorkDuration"] = df.set_index('Pharmacy_id')["Work period, hours"]

# Calculate EndTime in hours
df['EndTime'] = (df['OpenTime'] + df['Work period, hours'])-1 % 24  # Use modulo 24 for wrap-around

# Create the desired dictionary
pharmacy_dict = {str(row['Pharmacy_id']): (row['OpenTime'], row['EndTime']) for _, row in df.iterrows()}

### 4.2.2. Calculation of distance between pharmacies

In [7]:
import math
from itertools import combinations

# Iterate over the list of numeric columns to replace commas with dots and convert the data type to float.
numeric_columns = ['latitude', 'longitude']  # List of columns to convert to numeric
#for column in numeric_columns:
#    df[column] = df[column].str.replace(',', '.').astype(float)
coordinates = df[['Pharmacy_id', 'latitude', 'longitude']]
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])    # Convert latitude and longitude from degrees to radians

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    R = 6371.0          # Radius of Earth in kilometers (mean radius = 6,371 km)
    return R * c

distance_dict = {}      # Create a dictionary to store distances
# Compute distances between each pair of pharmacies
for (id1, lat1, lon1), (id2, lat2, lon2) in combinations(coordinates.itertuples(index=False), 2):
    distance = haversine(lat1, lon1, lat2, lon2)
    distance_dict[(id1, id2)] = distance
    distance_dict[(id2, id1)] = distance  # Add the reverse pair since distance is symmetric

# Convert distances into a format suitable for AMPL
# AMPL expects a dictionary where each key is a tuple (p, pp) and the value is the distance
ampl.param['ph_Distance'] = { (p, pp): distance for (p, pp), distance in distance_dict.items() }

### 4.3. Load data from Employees.csv

In [8]:
df_empl = pd.read_csv(employees_file, delimiter = ";", keep_default_na = False)

# # Define a set in AMPL named 'EMPLOYEES', containing unique employee IDs from the 'Employee_id' column in the DataFrame.
ampl.set['EMPLOYEES'] = set(df_empl['Employee_id'])
ampl.set['EMPL_WORKPLACE'] = df_empl.groupby('Employee_id')['Pharmacy_id'].apply(set).to_dict()

# Define the parameters for employees in AMPL.
# Specify which columns should be numeric; in this case, 'SoldPerHour'.
#numeric_columns = ['SoldPerHour']   # List of columns to convert to numeric.
#for column in numeric_columns:      # Iterate over the list of numeric columns to replace commas with dots and convert the data type to float.
#   df_empl[column] = df_empl[column].str.replace(',', '.').astype(float)

# Initialize a dictionary to store the parameter data for emplAtr
emplAtr_dict = {}

# Loop through each employee and populate the emplAtr dictionary for each parameter
for index, row in df_empl.iterrows():
    employee_id = row['Employee_id']
    emplAtr_dict[(employee_id, 'MaxReplaceDistance')] = row['ReplaceDistance']
    emplAtr_dict[(employee_id, 'MaxItemsSoldPerHour')] = row['SoldPerHour']
    emplAtr_dict[(employee_id, 'SalaryPerHour')] = row['SalaryPerHour']
    emplAtr_dict[(employee_id, 'VacationStartPeriod')] = row['VacationStartPeriod']
    emplAtr_dict[(employee_id, 'VacationDuration')] = row['VacationDuration']
    emplAtr_dict[(employee_id, 'SalesPerformance')] = row['SalesPerformance']
# Now pass this dictionary to the AMPL parameter `emplAtr`
ampl.param['emplAtr'] = emplAtr_dict

### 4.4. Define the operating mode data

In [9]:
# Example data
data = {
    'ShiftType': ["Base", "Base+1", "Base+2", "Rotational", "Additional", "Additional+1","Additional+2"],  # Different shift types
    'DailyWorkHours': [8, 8, 8, 12, 4, 4, 4],                   # Daily work hours for each shift type
    'OvertimeHours': [0, 1, 2, 0, 0, 1, 2],                     # Overtime hours for each shift type
    'maxWeeklyHours': [40, 40, 40, 48, 4, 4, 4],                # Maximum weekly hours for each shift type
    'maxWeeklyOvertimeHours': [0, 6, 8, 0, 0, 6, 8],            # Maximum weekly overtime hours
    'maxYearOvertimeHours': [0, 96, 96, 0, 0, 96, 96]           # Maximum yearly overtime hours
}

# Convert data to DataFrame
df_shift = pd.DataFrame(data)

# Calculate the total hours
df_shift['TotalHours'] = df_shift['DailyWorkHours'] + df_shift['OvertimeHours']

# Convert the TotalHours column to a numpy array
total_hours_array = df_shift['TotalHours'].to_numpy()

# Set SHIFT_TYPES in AMPL with the unique shift types from the DataFrame
ampl.set['SHIFT_TYPES'] = set(df_shift['ShiftType'])

# Initialize the dictionary for shiftAtr
shiftAtr_dict = {}

# Populate the dictionary
for index, row in df_shift.iterrows():
    shift_type = row['ShiftType']
    shiftAtr_dict[(shift_type, 'DailyWorkHours')] = row['DailyWorkHours']
    shiftAtr_dict[(shift_type, 'OvertimeHours')] = row['OvertimeHours']
    shiftAtr_dict[(shift_type, 'maxWeeklyHours')] = row['maxWeeklyHours']
    shiftAtr_dict[(shift_type, 'maxWeeklyOvertimeHours')] = row['maxWeeklyOvertimeHours']
    shiftAtr_dict[(shift_type, 'maxYearOvertimeHours')] = row['maxYearOvertimeHours']
    shiftAtr_dict[(shift_type, 'TotalHours')] = row['TotalHours']

# Set the parameter in AMPL
ampl.param['shiftAtr'] = shiftAtr_dict

total_hours_output = {}

# Iterate through the data dictionary
for (key1, key2), value in shiftAtr_dict.items():
    # Check if the second key is 'TotalHours'
    if key2 == 'TotalHours':
        # Store the result in the output dictionary
        total_hours_output[key1] = value

# Define staffReserve
ampl.param['staffReserve'] = 1.1

### 4.5. Define the availability table for the daily schedules

In [10]:
from math import factorial

# Function to compute the availability table where employees must work d consecutive hours
def schedule(start, end, d):
    w = end - start  # Total number of working hours for the pharmacy (shift length)
    s = [[0 for _ in range(24)] for _ in range(w - d + 1)]

    for i in range(w - d + 1):
        for dd in range(d):
            s[i][start + i + dd] = 1
            return s

# Initialize SHIFT_HOURS and ph_DaySchedule
SHIFT_HOURS = []
ph_DaySchedule = {}  # Should be a dictionary for easy access

# Iterate over each pharmacy and generate the availability schedule
for pharmacy_name, (start_hour, end_hour) in pharmacy_dict.items():
    shift_length = end_hour - start_hour
    #print(f"\n{pharmacy_name} (Working hours: {start_hour}:00 - {end_hour}:00, Shift length: {shift_length} hours)")

    # Iterate over each shift type and corresponding work hours in total_hours_output
    for shift_type_, work_hours in total_hours_output.items():
        # Skip the entry if work_hours is 0
        if work_hours == 0:
            continue  # Skip to the next iteration

        # Ensure the employee's consecutive hours are less than or equal to the pharmacy's total working hours
        if work_hours <= shift_length:
            #print(f"\nPossible schedules for {work_hours} consecutive working hours under shift type {shift_type_}:")

            # Generate the availability schedule for this combination
            availability_schedule = schedule(start_hour, end_hour, work_hours)

            # Populate SHIFT_HOURS and ph_DaySchedule
            for schedule_index, schedule_row in enumerate(availability_schedule):

                # Create a tuple for each schedule: (pharmacy, shift_type, start_hour)
                for i in range(work_hours):
                    schedule_tuple = (int(pharmacy_name), shift_type_, start_hour + schedule_index, start_hour + schedule_index + i)
                    SHIFT_HOURS.append(schedule_tuple)

                    # For each hour of the day (0 to 23), assign 1 if employee is working, otherwise 0
                for hour in range(23):
                        # Only include hours that are relevant for the pharmacy's working hours
                    if start_hour + schedule_index <= hour < start_hour + schedule_index + work_hours:
                            # Assign 1 for hours the employee is working
                        ph_DaySchedule[(int(pharmacy_name), shift_type_, start_hour + schedule_index, hour)] = 1

# Assign the SHIFT_HOURS set
#ampl.set['SHIFT_HOURS'] = set(SHIFT_HOURS)

# Assign the ph_DaySchedule parameter
ampl.param['ph_DaySchedule'] = ph_DaySchedule

### 4.6. Define the availability table for the cycle schedule

In [11]:
%%ampl_eval
data;
# These data describe the operating rules for different schedules within cycle
param cycleSchedule: 1   2   3   4   5   6   7   8   9   10  11  12 13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30:=
"Base"              1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
"Base+1"            1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
"Base+2"            1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
"Rotational"        1   1   0   0   1   1   0   0   1   1   0   0   1   1   0   0   1   1   0   0   1   1   0   0   1   1   0   0   1   1
"Additional"        1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
"Additional+1"      1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
"Additional+2"      1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1 ;

## 5. Solve problem

In [12]:
ampl.option['show_stats'] = 1         # (1) Show statistical information about the size of the problem. Default 0 (statistics are not displayed)

ampl.option['mp_options'] = 'outlev=1 lim:time=300'
ampl.solve(solver='gurobi') # with Community Edition use 'highs' or 'cbc'


Presolve eliminates 17552 constraints and 98249 variables.
Substitution eliminates 60438 variables.
Adjusted problem:
130745 variables:
	127862 binary variables
	2883 linear variables
27424 constraints, all linear; 1063603 nonzeros
	27424 inequality constraints
1 linear objective; 129436 nonzeros.

Gurobi 11.0.1: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter TimeLimit to value 300
  lim:time = 300
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "TUXEDO OS 2")

CPU model: 13th Gen Intel(R) Core(TM) i5-1340P, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 27424 rows, 130746 columns and 1063603 nonzeros
Model fingerprint: 0x98fee74b
Variable types: 2884 continuous, 127862 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 2e+05]
  RHS range     

## 6. Display the solution

In [15]:
ampl.option['display_1col'] = 0       # Data Display Settings
ampl.option['omit_zero_rows'] = 1     # Hide rows with 0 values. Default (0)
ampl.option['omit_zero_col'] = 1      # Hide columns with 0 values. Default (0)

ampl.display('StartPeriodSched, StartDaySchedule, AssignDaySchedule')

StartPeriodSched [East,*,*,Additional,1] (tr)
:    33  34  35  36  37  38  47  48    :=
65    1   0   0   0   .   0   .   0
96    1   0   0   0   0   0   0   0
203   .   0   0   1   0   0   0   0
218   .   0   1   .   0   0   0   .
235   .   .   .   .   1   0   0   .

 [East,*,*,Additional+1,1] (tr)
:    33  34  35  36  37  38  47  48    :=

 [East,*,*,Additional+2,1] (tr)
:    33  34  35  36  37  38  47  48    :=

 [East,*,*,Base,1] (tr)
:    33  34  35  36  37  38  47  48    :=
25    0   0   0   1   0   0   0   0
66    0   0   0   0   .   0   .   1
67    0   0   1   0   .   0   .   0
68    0   0   1   0   0   0   0   0
88    1   0   0   0   0   0   0   0
89    1   0   0   0   0   0   0   0
90    0   0   0   0   0   0   1   0
91    0   0   1   0   0   0   0   0
97    1   0   0   0   0   0   0   0
106   0   0   0   0   1   0   0   0
107   0   0   0   1   0   0   0   0

 [East,*,*,Base+1,1] (tr)
:    33  34  35  36  37  38  47  48    :=
108   0   0   0   0   0   0   0   1
127   .   0   

### Solution for the Manager

![alt text](https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/StaffChain/Staff_1.jpg?raw=1)

### Solution for the Employees

![alt text](https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/StaffChain/Staff_2.jpg?raw=1)

## 7. Retrieve solution and export it to a *.xlsx file

In [ ]:
! pip install openpyxl
amplvar = dict()
# Create an ExcelWriter object for saving DataFrames to an Excel file at the specified path
with pd.ExcelWriter('staff_schedule.xlsx') as writer:
    # Generate a list of all variable names from the AMPL model
    list_of_ampl_variables = [item[0] for item in ampl.get_variables()]
    # Iterate over each variable name in the list
    for key_ampl in list_of_ampl_variables:
        # Skip certain variables that are not to be processed (these variables won't be included in the output)
        if key_ampl not in ['']:
            # Convert the AMPL variable data to a pandas DataFrame
            df = ampl.var[key_ampl].to_pandas()
            # Filter the DataFrame to include only rows where the variable's value is greater than a small threshold (10^-5)
            filtered_df = df[df[f"{key_ampl}.val"] > 10e-5]
            # Save the filtered DataFrame to the corresponding sheet in the Excel file
            if not filtered_df.empty:  # Ensure that only non-empty DataFrames are saved
                filtered_df.to_excel(writer, sheet_name=key_ampl, index=True)
                # Save the filtered DataFrame to a dictionary
                #amplvar[key_ampl] = filtered_df

## 8. Enhancements

* Study the problem and find usable approaches to reducing the number of model variables, develop additional preliminary heuristics to obtain a smaller model.
* Try to solve the model with a lot of data: [*Pharmacies_.csv*](Pharmacies_.csv), [*Employees_.csv*](Employees_.csv), [*Sales_.csv*](Sales_.csv).